In [112]:
import json
import os
from pathlib import Path
from typing import List, Tuple, Union

import numpy as np
import pandas as pd
from PIL import Image
import torch

from megapose.datasets.object_dataset import RigidObject, RigidObjectDataset
from megapose.utils.load_model import NAMED_MODELS, load_named_model
from megapose.inference.types import (
    DetectionsType,
    ObservationTensor,
    PoseEstimatesType,
)

from bokeh.io import export_png
from bokeh.plotting import gridplot
from PIL import Image

# MegaPose
from megapose.config import LOCAL_DATA_DIR
from megapose.datasets.object_dataset import RigidObject, RigidObjectDataset
from megapose.datasets.scene_dataset import CameraData, ObjectData
from megapose.inference.types import (
    DetectionsType,
    ObservationTensor,
    PoseEstimatesType,
)
from megapose.inference.utils import make_detections_from_object_data
from megapose.lib3d.transform import Transform
from megapose.panda3d_renderer import Panda3dLightData
from megapose.panda3d_renderer.panda3d_scene_renderer import Panda3dSceneRenderer
from megapose.utils.conversion import convert_scene_observation_to_panda3d
from megapose.utils.load_model import NAMED_MODELS, load_named_model
from megapose.utils.logging import get_logger, set_logging_level
from megapose.visualization.bokeh_plotter import BokehPlotter
from megapose.visualization.utils import make_contour_overlay

os.environ["MEGAPOSE_DATA_DIR"] = "/scratch/jeyan/megapose"

In [100]:
meshpath = Path("/scratch/jeyan/barreldata/models3d/barrelsingle-scaled-subdivide.ply")
rigobj = RigidObject(label="barrel", mesh_path=meshpath, mesh_units="m")
rigobjds = RigidObjectDataset([rigobj])

In [ ]:
model_name = "megapose-1.0-RGB-multi-hypothesis"
model_info = NAMED_MODELS[model_name]
pose_estimator = load_named_model(model_name, rigobjds).cuda()

In [117]:
imgdir = Path("/scratch/jeyan/barreldata/results/barrelddt1/mesh-renders-spotlight")
imgpaths = sorted(list(imgdir.glob("*.jpg")) + list(imgdir.glob("*.png")))
# (b, c, h, w)
imgs = np.array([np.array(Image.open(imgpath)) for imgpath in imgpaths]).transpose(0, 3, 1, 2) / 255.0
K = np.array([
    [1246.0, 0.0, 960.0],
    [0.0, 1246.0, 437.5],
    [0.0, 0.0, 1.0]
])
camdata = CameraData(K=K, resolution=(875, 1920))
K_batched = np.tile(K, (len(imgs), 1, 1))
observation = ObservationTensor(images=torch.from_numpy(imgs).float().cuda(), K=torch.from_numpy(K_batched).float().cuda())

maskdir = Path("/scratch/jeyan/barreldata/results/barrelddt1/masks")
maskpaths = sorted(list(maskdir.glob("*.png")))
masks = [np.array(Image.open(imgpath)) / 255.0 for imgpath in imgpaths]
bboxes = []
for i, mask in enumerate(masks):
    orig_mask_modal = np.array(Image.open(maskpaths[i])) / 255.0
    sumvert = np.sum(orig_mask_modal, axis=0)
    left = np.where(sumvert > 0)[0][0]
    right = np.where(sumvert > 0)[0][-1]
    sumhor = np.sum(orig_mask_modal, axis=1)
    bottom = np.where(sumhor > 0)[0][0]
    top = np.where(sumhor > 0)[0][-1]
    # bboxes should be [x1, y1, x2, y2] in the detections data type
    bbox = [left, top, right, bottom]
    bboxes.append(bbox)
bboxes = torch.from_numpy(np.array(bboxes)).cuda()

In [107]:
det_infos = pd.DataFrame(
    dict(
        label="barrel",
        batch_im_id=np.arange(len(imgpaths)),
        instance_id=0,
    )
)
detections = DetectionsType(infos=det_infos, bboxes=bboxes)

In [108]:
pose_infos = pd.DataFrame(
    dict(
        label="barrel",
        batch_im_id=np.repeat(np.arange(len(imgpaths)), 5),
        instance_id=0,
        hypothesis_id=np.tile(np.arange(5), len(imgpaths)),
    )
)

foundpose_estimate_path = Path("/scratch/jeyan/foundpose/output_barrel/inference/estimated-poses.json")
with open(foundpose_estimate_path, "rt") as f:
    foundpose_estimates = json.load(f)

poses = []
for i, est in enumerate(foundpose_estimates):
    R = est["R"]
    t = est["t"]
    T = np.eye(4)
    T[:3, :3] = R
    T[:3, 3] = np.array(t).reshape(-1)
    poses.append(T)
poses = torch.from_numpy(np.array(poses)).float().cuda()
coarse_poses = PoseEstimatesType(infos=pose_infos, poses=poses)

In [109]:
output, _ = pose_estimator.run_inference_pipeline(
    observation,
    detections=detections,
    run_detector=False,
    coarse_estimates=coarse_poses,
    **model_info["inference_parameters"]
)

In [131]:
def load_observation(
    example_dir: Path,
    load_depth: bool = False,
) -> Tuple[np.ndarray, Union[None, np.ndarray], CameraData]:
    camera_data = CameraData.from_json((example_dir / "camera_data.json").read_text())

    rgb = np.array(Image.open(example_dir / "image_rgb.png"), dtype=np.uint8)
    assert rgb.shape[:2] == camera_data.resolution

    depth = None
    if load_depth:
        depth = np.array(Image.open(example_dir / "image_depth.png"), dtype=np.float32) / 1000
        assert depth.shape[:2] == camera_data.resolution

    return rgb, depth, camera_data

def save_predictions(
    example_dir: Path,
    pose_estimates: PoseEstimatesType,
) -> None:
    labels = pose_estimates.infos["label"]
    poses = pose_estimates.poses.cpu().numpy()
    object_data = [
        ObjectData(label=label, TWO=Transform(pose)) for label, pose in zip(labels, poses)
    ]
    object_data_json = json.dumps([x.to_json() for x in object_data])
    output_fn = example_dir / "outputs" / "object_data.json"
    output_fn.parent.mkdir(exist_ok=True)
    output_fn.write_text(object_data_json)
    return

def load_object_data(data_path: Path) -> List[ObjectData]:
    object_data = json.loads(data_path.read_text())
    object_data = [ObjectData.from_json(d) for d in object_data]
    return object_data

def make_output_visualization(
    example_dir: Path, idx, rigobjds, rgb, camera_data
) -> None:

    camera_data.TWC = Transform(np.eye(4))
    object_datas = load_object_data(example_dir / "outputs" / "object_data.json")
    object_dataset = rigobjds

    renderer = Panda3dSceneRenderer(object_dataset)

    camera_data, object_datas = convert_scene_observation_to_panda3d(camera_data, object_datas)
    light_datas = [
        Panda3dLightData(
            light_type="ambient",
            color=((1.0, 1.0, 1.0, 1)),
        ),
    ]
    renderings = renderer.render_scene(
        object_datas[idx:idx+1],
        [camera_data],
        light_datas,
        render_depth=False,
        render_binary_mask=False,
        render_normals=False,
        copy_arrays=True,
    )[0]

    plotter = BokehPlotter()

    fig_rgb = plotter.plot_image(rgb)
    fig_mesh_overlay = plotter.plot_overlay(rgb, renderings.rgb)
    contour_overlay = make_contour_overlay(
        rgb, renderings.rgb, dilate_iterations=1, color=(0, 255, 0)
    )["img"]
    fig_contour_overlay = plotter.plot_image(contour_overlay)
    fig_all = gridplot([[fig_rgb, fig_contour_overlay, fig_mesh_overlay]], toolbar_location=None)
    vis_dir = example_dir / "visualizations"
    vis_dir.mkdir(exist_ok=True)
    export_png(fig_mesh_overlay, filename=vis_dir / "mesh_overlay.png")
    export_png(fig_contour_overlay, filename=vis_dir / "contour_overlay.png")
    export_png(fig_all, filename=vis_dir / "all_results.png")
    return

In [129]:
outdir = Path("/scratch/jeyan/megapose/examples/barrel")

save_predictions(outdir, output)

In [ ]:
rgb = (imgs[0].transpose(1, 2, 0) * 255).astype(np.uint8)
make_output_visualization(outdir, 0, rigobjds, rgb, camdata)

In [134]:
def make_detections_visualization(
    example_dir: Path, rgb, detections
) -> None:
    plotter = BokehPlotter()
    fig_rgb = plotter.plot_image(rgb)
    fig_det = plotter.plot_detections(fig_rgb, detections=detections)
    output_fn = example_dir / "visualizations" / "detections.png"
    output_fn.parent.mkdir(exist_ok=True)
    export_png(fig_det, filename=output_fn)
    return

In [ ]:
make_detections_visualization(outdir, rgb, detections[0:1])